In [1]:
from Model import *
import numpy as np

In [ ]:
m = Model()

m.multiplexing = 2
m.ploidy = 1
m.detection_probability = .2

# Array of probabilities p_2, p_1, p_0
p_2 = .1
p_1 = .2
p_0 = .5

print(m.predict((p_0, p_1, p_2)))

Test cases for the collapse_homologs method

In [ ]:
print("Test cases for the collapse_homologs method")
print("ploidy\talpha\tbeta\tprediction\tactual")
m.ploidy = 1
print("1\t\t1\t\t1\t\t" + str(m.collapse_homologs(1, 1)) + "\t\t" + str(p_2))
m.ploidy = 2
print("2\t\t2\t\t1\t\t" + str(m.collapse_homologs(2, 1)) + "\t\t" + str(p_2*p_1 * 2))
m.ploidy = 3
print("3\t\t2\t\t1\t\t" + str(m.collapse_homologs(2, 1)) + "\t\t" + str(p_2*p_1*p_0 * 6 + p_1**3))
print("3\t\t0\t\t3\t\t" + str(m.collapse_homologs(0, 3)) + "\t\t" + str(p_1**3))

Test cases for the detect method

In [ ]:
print("Test cases for the detect method")
print("detection_probability\tploidy\talpha\tbeta\tprediction\tactual")
m.detection_probability = 1
m.ploidy = 1
print("1\t\t\t\t\t\t1\t\t0\t\t0\t\t" + str(m.detect(0, 0)) + "\t\t\t" + str(m.collapse_homologs(0, 0)))
m.detection_probability = .5
m.ploidy = 2
print("0.5\t\t\t\t\t\t2\t\t1\t\t1\t\t" + str(m.detect(1, 1)) + "\t\t\t" + str(m.collapse_homologs(1, 1)*.5**2 + m.collapse_homologs(2, 1)*2*2*.5**3 + m.collapse_homologs(2, 2)*4*.5**4))
m.detection_probability = .5
m.ploidy = 2
print("0.5\t\t\t\t\t\t2\t\t2\t\t1\t\t" + str(m.detect(2, 1)) + "\t\t\t" + str(m.collapse_homologs(2, 1)*.5**3 + m.collapse_homologs(2, 2)*2*.5**4))

Test cases for the whole model

In [ ]:
m.multiplexing = 1
m.ploidy = 1
m.detection_probability = 1
print(m.predict((.1, .2, .5)))

Use a static model to predict distances between loci using in-silico GAM data
1. Generate the structure and view it

In [ ]:
from GAM import GAM
from utilities import *
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance as dist

structure = random_walk(length=1000)
structure = center_structure(structure)
pickle.dump(structure, open('teststructure.pkl', 'wb'))

fig = plt.figure()
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)
ax.projection = '3d'
ax.plot(*structure.T)

2. Set up the experiment and run it

In [ ]:
nuclear_radius = np.max(np.linalg.norm(structure, axis=1))
slice_width = nuclear_radius * .5

g = GAM(slice_width=slice_width, pick_slice=GAM.uniform_radius, nuclear_radius=nuclear_radius)
run = g.run(["myrandomwalk.pkl"], NPs=10000)

3. View the results

In [ ]:
fig, [ax1, ax2] = plt.subplots(1,2)

ax1.imshow(run["results"]["cosectioning_frequency"], cmap='viridis')
ax1.set_title("Cosectioning map")
ax2.imshow(dist.cdist(structure, structure))
ax2.set_title("Distance map")

4. Use a static model to predict the distances between beads i and j

In [ ]:
s = StaticModel(slice_range=2 * nuclear_radius + slice_width, slice_width=slice_width)
beads = 100

fits = np.zeros((beads, beads))

for i in range(beads):
    for j in range(i + 1, beads):
        m_i = (run["results"]["m_i"][0][i][j], run["results"]["m_i"][1][i][j], run["results"]["m_i"][2][i][j])
        fits[i,j] = s.fit(m_i, (1,), cost=Model.default_cost).x

In [ ]:
fig, ax = plt.subplots()

ax.plot(np.arange(len(structure)), run["results"]["sectioning_frequency"])
plt.show()

fig = plt.figure()
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)
ax.projection = '3d'
a = ax.scatter(*structure.T, c=run["results"]["sectioning_frequency"])
fig.colorbar(a, ax=ax)

In [ ]:
from matplotlib.colors import Normalize
from matplotlib import cm

prox = dist.cdist(structure[:beads], structure[:beads])

cmap = cm.viridis
vmin, vmax = 0, np.max([fits, prox])  # Replace with your desired bounds

# Normalize the data to the range [0, 1]
norm = Normalize(vmin=vmin, vmax=vmax)

fig, [ax1, ax2] = plt.subplots(1,2)

a=ax1.imshow(fits, norm=norm)
fig.colorbar(a, ax=ax1, shrink=.5)
a=ax2.imshow(prox, norm=norm)
fig.colorbar(a, ax=ax2, shrink=.5)

In [ ]:
func = np.vectorize(s.fit)
func(run["results"]["m_i"], (1,))

Test the slice model

In [5]:
sl = SLICE((.5, .2, .1), (.2, .3, .2), ploidy=2)

print(sl.predict(1))

m = Model(ploidy=2)
print(m.predict((.2, .3, .2)))

(0.04000000000000001, 0.42, 0.54)
(0.04000000000000001, 0.42, 0.54)
